## Building Machine Learning Classifiers: Random Forest on a holdout test set

### Read in  & clean text

In [1]:
import nltk
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data  = pd.read_csv('SMSSpamCollection.tsv', sep='\t', header=None)
data.columns = ['Label', 'body_text']

#creating a new feature
def count_puct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(' ')),3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(' '))
data['punct%']= data['body_text'].apply(lambda x: count_puct(x))

#removing punctuations, stopwords, applying tokenization and stemming
def clean_text(text):
    text = ''.join([word.lower() for word in text if word not in string.punctuation])
    tokenize = re.split('\W+', text)
    text = [word for word in tokenize if word not in stopwords]
    text = [ps.stem(word) for word in text]
    return text

#converting the text part of the data to vectors using tfidf
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
x_tfidf = tfidf_vect.fit_transform(data['body_text'])

#concatinating it back to the numerical featres
features = pd.concat([data['body_len'],data['punct%'], pd.DataFrame(x_tfidf.toarray())], axis=1)
features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8097,8098,8099,8100,8101,8102,8103,8104,8105,8106
0,160,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Explore RandomForestClassifier through Holdout set

In [3]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [5]:
x_train, x_test, y_train, y_test = train_test_split(features, data['Label'], test_size=0.2)

In [6]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
model = rf.fit(x_train, y_train)

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:1673: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [9]:
sorted(zip(model.feature_importances_, x_train.columns), reverse=True)[0:10]

[(0.04604362339601162, 1804),
 (0.043165026005380705, 'body_len'),
 (0.03846610923273863, 3135),
 (0.03365310705250649, 2032),
 (0.030931223679858476, 7353),
 (0.02883608559971434, 4799),
 (0.02719479507827037, 5727),
 (0.024655279624479953, 6288),
 (0.021477941382854304, 1361),
 (0.014465667171416561, 7030)]

In [11]:
y_pred = model.predict(x_test)

precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

C:\Users\HP\anaconda3\lib\site-packages\sklearn\utils\validation.py:1673: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [12]:
print(f'precision: {precision}/Recall: {recall}/')

precision: 1.0/Recall: 0.5490196078431373/
